<b> <font size = 5> Intersection count and distance to highway data from Open Street Maps </b> </font>

In this iPython notebook, the Overpass API from Open Street Maps is used to determine the location of all traffic signals within a given bounding box. The Overpy library is used to send the request to the API and this call returns the latitude and longitude of all traffic signals. Next, the distance between each traffic intersection and each point in the monitoring data is measured. A traffic score is calculated as the 'Number of traffic intersections within a 1,000 ft buffer' to each point in the monitoring data

The second section of this notebook uses the Overpass API to get the latitude and longitude of all points within a bounding box classified as a highway. Next, the distance from each monitoring location to the closest highway is determined. 

In [5]:
#Import python packages including overpy
import overpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import nearest_points

import seaborn as sns

from mpl_toolkits.axes_grid1 import make_axes_locatable

import math

import time


from scipy.stats import boxcox


from matplotlib import cm

import matplotlib.lines as mlines

sns.set(style = 'whitegrid')
sns.set_palette('bright')
%matplotlib inline

# <b> <font size = 5> Fetch all nodes using the API Query. Here the node is specified as 'Highway=traffic_signals' </b> </font> 

In [7]:
#Call overpass API and pass bounding box. 
api = overpy.Overpass()
result = api.query("""
    node(37.68,-122.36,37.8712,-122.03) ["highway"="traffic_signals"];
    (._;>;);
    out body;
    """)
traffic_lat = []
traffic_lon = []
for node in result.nodes:
    traffic_lat.append(node.lat)
    traffic_lon.append(node.lon)


In [9]:
#Write Latitude and Longitude data to a dataframe
traffic_df = pd.DataFrame(list(zip(traffic_lat, traffic_lon)), columns = ['Latitude', 'Longitude'])

##### Write to csv
traffic_df.to_csv("Data/Raw-data/all_traffic_intersections.csv")

# <b> <font size = 5>  Load traffic intersection data</b> </font>

In [11]:
traffic_df = pd.read_csv("Data/Raw-data/all_traffic_intersections.csv")

In [12]:
#Drop the first column
traffic_df.drop(columns = ['Unnamed: 0'], inplace=True)

In [13]:
## Rename index and intersection number
traffic_df.rename(columns = {'index':'Intersection'}, inplace=True)

In [14]:
### Add an empty column for distance
traffic_df['dist'] = 0
traffic_df['dist'].astype(float)

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1631    0.0
1632    0.0
1633    0.0
1634    0.0
1635    0.0
Name: dist, Length: 1636, dtype: float64

## <b> <font size  = 4> Convert traffic dataset into a column format to calculate distance </b> </font>

In [ ]:
# Create individual dataframes
traffic_lat = traffic_df[['Intersection', 'Latitude']]
traffic_long = traffic_df[['Intersection', 'Longitude']]
traffic_dist = traffic_df[['Intersection', 'dist']]

In [ ]:
# Transpose all the dataframes
traffic_lat = traffic_lat.T
traffic_long = traffic_long.T
traffic_dist  = traffic_dist.T

In [ ]:
## Make the header as the first row in each transposed dataframe
traffic_lat = traffic_lat.rename(columns=traffic_lat.iloc[0].astype(int)).drop(traffic_lat.index[0])
traffic_long = traffic_long.rename(columns=traffic_long.iloc[0].astype(int)).drop(traffic_long.index[0])
traffic_dist = traffic_dist.rename(columns=traffic_dist.iloc[0].astype(int)).drop(traffic_dist.index[0])

In [ ]:
## Add suffix to column header based on the dataframe type
traffic_lat.columns = [str(col) + '_latitude' for col in traffic_lat.columns]
traffic_long.columns = [str(col) + '_longitude' for col in traffic_long.columns]
traffic_dist.columns = [str(col) + '_distance' for col in traffic_dist.columns]

In [ ]:
## Remove index for each dataframe
traffic_lat.reset_index(drop=True, inplace=True)
traffic_long.reset_index(drop=True, inplace=True)
traffic_dist.reset_index(drop=True, inplace=True)

In [ ]:
### Combine individual dataframes into one
traffic_combined = traffic_lat.join(traffic_long).join(traffic_dist)

In [ ]:
### Sort based on column names
traffic_combined = traffic_combined.reindex(columns=sorted(traffic_combined.columns))

In [ ]:
#Update dataframe to contain 21488 rows
traffic_combined = traffic_combined.loc[traffic_combined.index.repeat(21488)].reset_index(drop=True)

# <b> <font size = 5> Load Air Pollution Monitoring Data </b> </font>

In [2]:
df = pd.read_csv('EDF_Data.csv', header = 1)
df.tail()

,Longitude,Latitude,NO Value,NO2 Value,BC Value
21483,-122.034943,37.560076,129.999995,44.778220,3.923761
21484,-122.034724,37.560164,60.799998,39.027545,1.408693
21485,-122.034681,37.559830,34.622951,28.816797,2.659885
21486,-122.034504,37.559958,74.764705,35.735434,1.776353
21487,-122.034503,37.559957,78.754782,41.062757,2.014664


In [3]:
BC_df = df[['Longitude', 'Latitude', 'BC Value']]

In [4]:
NO2_df = df[['Longitude', 'Latitude', 'NO2 Value']]

## <b> <font size = 4> Combine BC and NO2 datasets with traffic data </b> </font> 

In [ ]:
combined_BC_traffic = BC_df.join(traffic_combined)

In [ ]:
combined_NO2_traffic = NO2_df.join(traffic_combined)

In [ ]:
combined_BC_traffic.head()

## <b> <font size = 4> Calculate distance between monitoring location and each traffic intersection </b> </font> 

**We only calculate the distance from each monitoring location in the BC dataset with traffic intersections since the location of measurements are the same for NO2 and BC**

In [ ]:
# Convert distance or emissions distance column to float type
for idx, col in enumerate(combined_BC_traffic.columns):
        if "_dist" in col:
            combined_BC_traffic[col] = pd.to_numeric(combined_BC_traffic[col], downcast="float")



In [24]:
### Defining a function to calculate the distance between two GPS coordinates (latitude and longitude)
def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d


In [ ]:
time1 = time.time()
for index, row in combined_BC_traffic.iterrows():
    for idx, col in enumerate(combined_BC_traffic.columns):
        if "_dist" in col:
            combined_BC_traffic.at[index,col] = float(distance((row.iloc[1], row.iloc[0]), (row.iloc[idx+1], row.iloc[idx+2])))*3280.84
            #BC_Facility.at[index,col] = float(row.iloc[idx])
time2 = time.time()            
    
print(time2 - time1)

In [ ]:
combined_BC_traffic.head()

##### Write the entire dataset to a csv file
combined_BC_traffic.to_csv("Data/Unused-data/BC_traffic_full.csv")

# <b> <font size = 5> Read Traffic Distance Data </b> </font>

In [ ]:
#Read dataset
combined_BC_traffic = pd.read_csv("Data/Unused-data/BC_traffic_full.csv")

In [ ]:
#Drop the latitude column
combined_BC_traffic = combined_BC_traffic[combined_BC_traffic.columns.drop(list(combined_BC_traffic.filter(regex='_latitude')))]

In [ ]:
#Drop the longitude column
combined_BC_traffic = combined_BC_traffic[combined_BC_traffic.columns.drop(list(combined_BC_traffic.filter(regex='_longitude')))]

In [ ]:
#Drop BC value
combined_BC_traffic = combined_BC_traffic[combined_BC_traffic.columns.drop(list(combined_BC_traffic.filter(regex='BC Value')))]

In [ ]:
#Clean-up the columns
combined_BC_traffic.drop(columns = ['Unnamed: 0'], inplace=True)

In [ ]:
#Write to a new csv file
combined_BC_traffic.to_csv("Data/Unused-data/BC_traffic_distance.csv")

## <b> <font size = 4> Count the number of intersections with distance <1,000 feet </b> </font>

In [17]:
#Read csv file
combined_BC_traffic = pd.read_csv("Data/Unused-data/BC_traffic_distance.csv")

In [18]:
#Create an empty column for number of intersection
combined_BC_traffic['number_intersections'] = 0

## <b> <font size = 4>  Define function using range that returns True or False if a value is between 0 - 1,000 feet. </b> </font>

In [ ]:
def count_values_in_range(series, range_min, range_max):

    # "between" returns a boolean Series equivalent to left <= series <= right.
    # NA values will be treated as False.
    return series.between(left=range_min, right=range_max).sum()

range_min, range_max = 0, 1000


combined_BC_traffic['number_intersections'] = combined_BC_traffic.apply(
    func=lambda row: count_values_in_range(row, range_min, range_max), axis=1)


In [ ]:
#Get only the rows that returned true
BC_traffic_score = combined_BC_traffic[['Latitude','Longitude','number_intersections']]

#### Write to a csv file
BC_traffic_score.to_csv("Data/Traffic_score_2000.csv")

# <b> <font size = 5> Calculate Distance to Closest Highway </b> </font>

<font size = 5> <b> Fetch all nodes using the API Query. Here the node is specified as 'Highway=motorway' </b> </font>

In [5]:
api = overpy.Overpass()

# fetch all ways and nodes
result = api.query("""
    way(37.68,-122.36,37.752,-122.130) ["highway" = "motorway"];
    (._;>;);
    out body;
    """)


highway_lat = []
highway_lon = []
for node in result.nodes:
    highway_lat.append(node.lat)
    highway_lon.append(node.lon)


In [6]:
highway_df = pd.DataFrame(list(zip(highway_lat, highway_lon)), columns = ['Latitude', 'Longitude'])

In [7]:
highway_df.reset_index(inplace=True)

In [8]:
highway_df.rename(columns = {'index':'Location_id'}, inplace=True)

##### Write to csv
highway_df.to_csv("Data/highway_locations.csv")

## <b> <font size  = 4>  Find location of closest highway to each point and measure distance </b> </font>

In [10]:
geometry_BC = [Point(xy) for xy in zip(BC_df['Longitude'], BC_df['Latitude'])]
geometry_NO2 = [Point(xy) for xy in zip(NO2_df['Longitude'], NO2_df['Latitude'])]
geometry_highway = [Point(xy) for xy in zip(highway_df['Longitude'], highway_df['Latitude'])]

In [11]:
crs = {'init': 'epsg:4326'}

In [12]:
# Create a geopandas dataframe with the coordinate reference system as epsg4326
geo_df_BC = gpd.GeoDataFrame(BC_df, crs = crs, geometry = geometry_BC)
geo_df_NO2 =gpd.GeoDataFrame(NO2_df, crs = crs, geometry = geometry_NO2)
geo_df_highway =gpd.GeoDataFrame(highway_df, crs = crs, geometry = geometry_highway)

**Use geopandas nearest function to get the location of the nearest highway from each monitoring location**

In [13]:
# Unary Union of the geo_df geometry

pts = geo_df_highway.geometry.unary_union
def near(point, pts=pts):
     # find the nearest point and return the corresponding Location
     nearest = geo_df_highway.geometry == nearest_points(point, pts)[1]
     return geo_df_highway[nearest]['Location_id'].to_numpy()[0]
geo_df_BC['Nearest_Highway'] = geo_df_BC.apply(lambda row: near(row.geometry), axis=1)

In [14]:
# Unary Union of the geo_df geometry

pts = geo_df_highway.geometry.unary_union
def near(point, pts=pts):
     # find the nearest point and return the corresponding Location
     nearest = geo_df_highway.geometry == nearest_points(point, pts)[1]
     return geo_df_highway[nearest]['Location_id'].to_numpy()[0]
geo_df_NO2['Nearest_Highway'] = geo_df_NO2.apply(lambda row: near(row.geometry), axis=1)

In [16]:
BC_df_highway = BC_df.merge(highway_df, left_on=['Nearest_Highway'], right_on = ['Location_id'], suffixes = ['_BC','_highway'])

In [17]:
BC_df_highway.head()

,Longitude_BC,Latitude_BC,BC Value,geometry_BC,Nearest_Highway,Location_id,Latitude_highway,Longitude_highway,geometry_highway
0,-122.322594,37.806781,0.818032,POINT (-122.32259 37.80678),221,221,37.7529546,-122.2073471,POINT (-122.20735 37.75295)
1,-122.322310,37.806150,0.551475,POINT (-122.32231 37.80615),221,221,37.7529546,-122.2073471,POINT (-122.20735 37.75295)
2,-122.322301,37.806420,0.593712,POINT (-122.32230 37.80642),221,221,37.7529546,-122.2073471,POINT (-122.20735 37.75295)
3,-122.322299,37.805880,0.489898,POINT (-122.32230 37.80588),221,221,37.7529546,-122.2073471,POINT (-122.20735 37.75295)
4,-122.322267,37.806689,0.739341,POINT (-122.32227 37.80669),221,221,37.7529546,-122.2073471,POINT (-122.20735 37.75295)


In [18]:
BC_df_highway.drop(columns = ['Location_id', 'geometry_BC','geometry_highway', 'Nearest_Highway'], inplace=True)

In [19]:
### Add an empty column for distance
BC_df_highway['dist'] = 0
BC_df_highway['dist'].astype(float)

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
21483    0.0
21484    0.0
21485    0.0
21486    0.0
21487    0.0
Name: dist, Length: 21488, dtype: float64

In [20]:
#Convert all distance columns to type float
BC_df_highway['dist'] = pd.to_numeric(BC_df_highway['dist'], downcast="float")
BC_df_highway['Latitude_highway'] = pd.to_numeric(BC_df_highway['Latitude_highway'], downcast="float")
BC_df_highway['Longitude_highway'] = pd.to_numeric(BC_df_highway['Longitude_highway'], downcast="float")


In [21]:
BC_df_highway.head()

,Longitude_BC,Latitude_BC,BC Value,Latitude_highway,Longitude_highway,dist
0,-122.322594,37.806781,0.818032,37.752956,-122.207344,0.0
1,-122.322310,37.806150,0.551475,37.752956,-122.207344,0.0
2,-122.322301,37.806420,0.593712,37.752956,-122.207344,0.0
3,-122.322299,37.805880,0.489898,37.752956,-122.207344,0.0
4,-122.322267,37.806689,0.739341,37.752956,-122.207344,0.0


In [22]:
BC_df_highway['Latitude_highway'].describe()

count    21488.000000
mean        37.741249
std          0.019706
min         37.684822
25%         37.740280
50%         37.752956
75%         37.752956
max         37.753536
Name: Latitude_highway, dtype: float64

**Apply the distance function previously defined to calculate the distance between the latitude and longitude of monitoring location, and latitude and longitude of closest highway**

In [25]:
BC_df_highway['Dist'] = BC_df_highway.apply(lambda row : distance((row['Latitude_BC'], row['Longitude_BC']), 
                                                       (row['Latitude_highway'], row['Longitude_highway'])), axis = 1) 

In [28]:
BC_df_highway['Dist'].describe()

count    21488.000000
mean         5.360958
std          4.145873
min          0.000029
25%          1.338816
50%          5.342147
75%          9.215371
max         16.222368
Name: Dist, dtype: float64

##### Write to a csv
BC_df_highway.to_csv("Data/BC_dist_highway.csv")

In [30]:
NO2_df_highway = NO2_df.merge(highway_df, left_on=['Nearest_Highway'], right_on = ['Location_id'], suffixes = ['_NO2','_highway'])

In [31]:
NO2_df_highway.drop(columns = ['Location_id', 'geometry_NO2','geometry_highway', 'Nearest_Highway'], inplace=True)

In [32]:
NO2_df_highway['Latitude_highway'] = pd.to_numeric(NO2_df_highway['Latitude_highway'], downcast="float")
NO2_df_highway['Longitude_highway'] = pd.to_numeric(NO2_df_highway['Longitude_highway'], downcast="float")


**Apply the distance function previously defined to calculate the distance between the latitude and longitude of monitoring location, and latitude and longitude of closest highway**

In [33]:
NO2_df_highway['Dist'] = NO2_df_highway.apply(lambda row : distance((row['Latitude_NO2'], row['Longitude_NO2']), 
                                                       (row['Latitude_highway'], row['Longitude_highway'])), axis = 1) 

In [35]:
NO2_df_highway['Dist'].describe()

count    21488.000000
mean         5.360958
std          4.145873
min          0.000029
25%          1.338816
50%          5.342147
75%          9.215371
max         16.222368
Name: Dist, dtype: float64

##### Write to csv
NO2_df_highway.to_csv("Data/NO2_dist_highway.csv")

# Reference

1.  Fetch location of traffic signals: <a> "https://python-overpy.readthedocs.io/en/latest/introduction.html"> </a>